In [7]:
from openai import OpenAI
from pinecone import Pinecone
from dotenv import load_dotenv
import os
load_dotenv()

openai_client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_BASE")
)

user_question = "what is possible with embroidery in HCI textiles research?"

# use OpenAI API text-embedding-3-small to get embeddings for a question
response = openai_client.embeddings.create(
    input=user_question,
    model="text-embedding-3-small" 
)
question_vector = response.data[0].embedding

# use Pinecone to search for the most relevant papers
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
index_name = "textiles-hci-01" 
index = pc.Index(index_name)

# search for the most relevant papers
# index.query(vector=[0.1, 0.2, 0.3], top_k=10, namespace='my_namespace')
query_results = index.query(
    top_k=5, # top 5 most relevant papers
    vector=question_vector, 
    includeValues=True,
    includeMetadata=True,
    namespace="pdf"
    # filter: { genre: { '$eq': 'action' }}
    )  

# get the metadata of the most relevant papers
# print(query_results)
# contexts = [item['metadata']['abstract'] for item in query_results['matches']]
contexts = [item['metadata']['text'] for item in query_results['matches']]
dois = [item['metadata']['doi'] for item in query_results['matches']]
print(dois)

# combine the contexts and the question
context_combined = "\n".join(contexts)
chat_input = f"Context: {context_combined}\nQuestion: {user_question}"

response = openai_client.chat.completions.create(
  model="gpt-3.5-turbo", 
  messages=[
    {"role": "system", "content": "You are a helpful research assistant and an expert in textile research and HCI research."},
    {"role": "user", "content": chat_input}
  ],
#   prompt=chat_input,
  max_tokens=1000 # limit the response to 1000 tokens
)

# get the answer
answer_content = response.choices[0].message.content  # Access the 'content' attribute of the message
print(answer_content)

/Users/pengyuecheng/Documents/UW/TECHIN 510/final/.venv/lib/python3.9/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


PineconeConfigurationError: You haven't specified an Api-Key.

In [6]:
import requests

# 替换以下 URL 为你的 Cloud Function 的实际 URL
# url = "https://showmessage-lebwgsie4q-uc.a.run.app"
url = "https://texverse-ai-qa-lebwgsie4q-uc.a.run.app"

# 替换这里的查询参数为你的问题
params = {'text': 'what is possible with knitting in HCI textiles research?'}

response = requests.get(url, params=params)

# 输出响应内容
print(response.text)

500 Internal Server Error: The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.


In [15]:
response.text

''

In [5]:
dois = ["https://doi.org/10.1145/3559400.3562001", "https://doi.org/10.1145/3559400.3562002", "https://doi.org/10.1145/3559400.3562003", "https://doi.org/10.1145/3559400.3562004", "https://doi.org/10.1145/3559400.3562005"]
answer = "1213"
answer = f"{answer}\n\n ### DOIs of the most relevant papers:\n" + "\n - ".join(dois)
answer

'1213\n\n ### DOIs of the most relevant papers:\nhttps://doi.org/10.1145/3559400.3562001\n - https://doi.org/10.1145/3559400.3562002\n - https://doi.org/10.1145/3559400.3562003\n - https://doi.org/10.1145/3559400.3562004\n - https://doi.org/10.1145/3559400.3562005'